In [ ]:
### install packages as needed

# Install a pip package in the current Jupyter kernel
# !{sys.executable} -m pip install ipykernel
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install seaborn
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install torch==1.9.1+cu102 torchvision==0.10.1+cu102 torchaudio===0.9.1 -f https://download.pytorch.org/whl/torch_stable.html
# !{sys.executable} -m pip install pyyaml
# !{sys.executable} -m pip install opencv-python
%load_ext blackcellmagic


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path
import glob
import yaml
from datetime import date
from scipy import ndimage
from PIL import Image
import matplotlib.pyplot as plt
import cv2 as cv
import data

# import pingouin as pg # for check homoskedacity (inequal variances)

params = {
    "project_stub": "Potato_Fertilizer_Othello",
    "data_acquisition_date": "Jun22_2020",
    "NDVI_threshold": 0.5,
    "data_import_path": Path.cwd() / "data" / "raw" / "Jun22_2020",
    "data_export_path": Path.cwd() / "data" / "processed" / "Jun22_2020",
    "plot_export_path": Path.cwd() / "image_export" / "Jun22_2020",
    "ground_truth_path": Path.cwd() / "data" / "raw" /'ground_truth.csv'
}

ground_truth = pd.read_csv(params["ground_truth_path"])

# export the params file for reference
with open(Path.cwd() / "config" / (str(date.today()) + "_" + params["data_acquisition_date"] + "_"  + "params.yaml"), "w") as file:
    documents = yaml.dump(params, file)

In [15]:
### useful functions
def rotate_and_crop(img, rot_angle, y_limits, x_limits):
    """takes an image and applies a rotation, then returns a subset of that image"""
    return ndimage.rotate(img, angle=rot_angle)[
        y_limits[0] : y_limits[1], x_limits[0] : x_limits[1]
    ]


def save_img(array, plot, index):
    """takes np.array and converts to a PIL.Image, then saves it into the data_export_path"""
    im = Image.fromarray(array.astype(np.uint8))
    im.save(
        params["data_export_path"] + "plot_" + str(plot) + "_index_" + index + ".png"
    )

def load_img(path):
    """open image file, replace all '-10000' transparent values with zero, and return"""
    image = cv.imread(path, cv.IMREAD_UNCHANGED)

    out_image = np.where(image < 0, 0, image)
    
    # out_image = np.array(image * 255., dtype=np.uint8)
    print(f"img min, max = {np.min(image), np.max(image)}, out min, max = {np.min(out_image), np.max(out_image)}")

    return out_image

def show_img(img):
    ''' plot array as img '''
    plt.figure(figsize = (8,30))
    plt.imshow(img, cmap="viridis")

In [16]:
### load images

raw_img_paths = sorted(glob.glob(os.path.join(params["data_import_path"], "*.tif")))
channel_order = ["blue-444", "blue", "green-531", "green", "nir", "red edge-705", "red edge-740", "red edge", "red-650", "red"]
field_image = np.stack([load_img(img_path) for img_path in raw_img_paths])

# field_image = np.array(image_list)
print(f"image_list shape:{field_image.shape}, type(field_image):{type(field_image)}")


img min, max = (-10000.0, 0.3787283), out min, max = (0.0, 0.3787283)
img min, max = (-10000.0, 0.40292442), out min, max = (0.0, 0.40292442)
img min, max = (-10000.0, 0.47094285), out min, max = (0.0, 0.47094285)
img min, max = (-10000.0, 0.37267798), out min, max = (0.0, 0.37267798)
img min, max = (-10000.0, 0.97884405), out min, max = (0.0, 0.97884405)
img min, max = (-10000.0, 0.64317596), out min, max = (0.0, 0.64317596)
img min, max = (-10000.0, 0.93435293), out min, max = (0.0, 0.93435293)
img min, max = (-10000.0, 0.6702947), out min, max = (0.0, 0.6702947)
img min, max = (-10000.0, 0.5182316), out min, max = (0.0, 0.5182316)
img min, max = (-10000.0, 0.56610835), out min, max = (0.0, 0.56610835)
image_list shape:(10, 11377, 4239), type(field_image):<class 'numpy.ndarray'>


In [18]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

ipywidgets.Text('Hello')